In [ ]:
%run ../../data/api_keys.ipynb
%run ../modulos.ipynb
%run ../funciones_temp.ipynb

In [3]:
def request_github(url,personal_access=personal_access_token):
    headers = {"Authorization": f"Bearer {personal_access}"}

    res=requests.get(url,headers=headers)
    return res.json()

In [28]:
def get_pages_repos(min_estrellas, max_estrellas, num_pagina=1):
    return f'https://api.github.com/search/repositories?per_page=100&page={num_pagina}&q=stars%3A{min_estrellas}..{max_estrellas}&sort=forks&order=desc'

In [29]:
headers = {"Authorization": f"Bearer {personal_access_token}","Accept": "application/vnd.github.v3.star+json","content-type": "application/json"}

In [138]:
client=pymongo.MongoClient('mongodb://localhost:27017/')
database=client['promisearch'] # database
collecion_repos_prueba=database['repositorios_prueba'] # colección

len(list(collecion_repos_prueba.find()))


2

## Inserciones a DB

In [30]:
def obtener_hist(user,repo,atributo,nom_col):

    consulta_url = f"https://api.github.com/repos/{user}/{repo}/{atributo}"
    repos_resp = []

    page=0
    response = request_github_no_json(consulta_url+"?per_page=100"+"&page="+str(page+1)).json() # primeras 100
    repos_resp.extend([re[nom_col] for re in response])

    while len(response)!=0:
        page+=1
        response = request_github_no_json(consulta_url+"?per_page=100"+"&page="+str(page+1)).json()
        repos_resp.extend([re[nom_col] for re in response])

  
    serie_acum=lista_fecha_a_df(repos_resp)
    cuantiles=pd.qcut(serie_acum, q=4, retbins=True)[1]
    cuant_prop=f'{cuantiles[1]},{",".join([str(round(ai,4)) for ai in list(cuantiles[2:] / cuantiles[1:-1])])}'

    return cuant_prop

In [31]:
def obtener_info_repo(user,repo):
    """Regresa json con info histórica y del repo"""

    star_count_url = "https://api.github.com/repos/"+user+"/"+repo

    response1 = request_github_no_json(star_count_url)
     
    cuantiles_estrellas = obtener_hist(user,repo,'stargazers','starred_at')
    cuantiles_forks = obtener_hist(user,repo,'forks','created_at')

    resp_json=response1.json()
    resp_json['cuant_estrellas']=cuantiles_estrellas
    resp_json['cuant_forks']=cuantiles_forks
 
    return resp_json

In [33]:
res_json=obtener_info_repo('pysentimiento','pysentimiento')
res_json

{'id': 292896321,
 'node_id': 'MDEwOlJlcG9zaXRvcnkyOTI4OTYzMjE=',
 'name': 'pysentimiento',
 'full_name': 'pysentimiento/pysentimiento',
 'private': False,
 'owner': {'login': 'pysentimiento',
  'id': 85897277,
  'node_id': 'MDEyOk9yZ2FuaXphdGlvbjg1ODk3Mjc3',
  'avatar_url': 'https://avatars.githubusercontent.com/u/85897277?v=4',
  'gravatar_id': '',
  'url': 'https://api.github.com/users/pysentimiento',
  'html_url': 'https://github.com/pysentimiento',
  'followers_url': 'https://api.github.com/users/pysentimiento/followers',
  'following_url': 'https://api.github.com/users/pysentimiento/following{/other_user}',
  'gists_url': 'https://api.github.com/users/pysentimiento/gists{/gist_id}',
  'starred_url': 'https://api.github.com/users/pysentimiento/starred{/owner}{/repo}',
  'subscriptions_url': 'https://api.github.com/users/pysentimiento/subscriptions',
  'organizations_url': 'https://api.github.com/users/pysentimiento/orgs',
  'repos_url': 'https://api.github.com/users/pysentimiento/

In [34]:
print(res_json.get('id',np.nan))
print(res_json.get('html_url',np.nan))
print(res_json.get('cuant_estrellas',np.nan))
print(res_json.get('cuant_forks',np.nan))
print(res_json.get('size',np.nan))
print(res_json.get('stargazers_count',np.nan))
print(res_json.get('forks',np.nan))
print(res_json.get('open_issues',np.nan))
print(res_json.get('subscribers_count',np.nan))
print(res_json.get('network_count',np.nan))

292896321
https://github.com/pysentimiento/pysentimiento
80.25,1.6386,1.2833,1.2741
7.5,1.7333,1.4231,1.2973
14289
215
24
4
7
24


In [35]:
min_estrellas = 500
max_estrellas = 600
numero_requests=1
def obtener_repos_de_pags(min_estrellas,max_estrellas,numero_requests,num_muestreo):
    repos_final = {}
    num_pagina=1 
    
    # Sólo agarrar muestreo de repos
    for _ in range(numero_requests):
        repos_bins = []

        i=0
        while i <1:
            # Construyo la URL
            url_objetivo = get_pages_repos(min_estrellas, max_estrellas, num_pagina)
            #print(num_pagina)
            print(url_objetivo)
            # Aqui hago la peticion a github
            search_github = requests.get(url_objetivo ,headers=headers)
            search_github_json = search_github.json()
            try:
                lista_repos = search_github_json['items'] # 100 repos
                lista_random = random.sample(lista_repos, num_muestreo)   
                print(f'Todo bien {_+1}')             
                repos_bins.extend(lista_random)
                
            except:  
                print('A dormir')
                time.sleep(61)
                i-=1
            i+=1
        num_pagina += 1
        
        repos_final[f'{_}'] = repos_bins
        # collecion_repos_prueba.insert_one({f'{_}':repos_bins})
    return repos_final

In [36]:
numero_requests=1
repos_final=obtener_repos_de_pags(600,700,numero_requests,50)

https://api.github.com/search/repositories?per_page=100&page=1&q=stars%3A600..700&sort=forks&order=desc
Todo bien 1


## Creación de csv

In [ ]:
id_repos = []
name_repos = []
sizes = []
stargazers_count_list = []
forks_list = []
open_issues_list = []
watchers = []
network_count_list = []
suscribers_list = []
repos1=[]

cuantiles_star=[]
cuantiles_forks=[]
lenguajes=[]

for k in tqdm_notebook(range(numero_requests)):
    # lista_dict=list(collecion_repos.find({ f'{k}' : { '$exists' : 'true' } }))[0][f'{k}']
    lista_dict=repos_final[f'{k}']
    for repo in tqdm_notebook(lista_dict, desc='Inner loop'):
        repo_name=repo['name']
        user=repo['owner']['login']
        # --------------------------------------------------
        json_repo=obtener_info_repo(user,repo_name)

        id_repos.append(json_repo.get('id',np.nan))
        name_repos.append(json_repo.get('html_url',np.nan))
        cuantiles_star.append(json_repo.get('cuant_estrellas',np.nan))
        cuantiles_forks.append(json_repo.get('cuant_forks',np.nan))
        lenguajes.append(json_repo.get('language',np.nan))
        sizes.append(json_repo.get('size',np.nan))
        stargazers_count_list.append(json_repo.get('stargazers_count',np.nan))
        forks_list.append(json_repo.get('forks',np.nan))
        open_issues_list.append(json_repo.get('open_issues',np.nan))
        watchers.append(json_repo.get('subscribers_count',np.nan))
        network_count_list.append(json_repo.get('network_count',np.nan)) # número de personas que han "contribuido"

In [156]:
repos_df = pd.DataFrame(dict(
    ids=id_repos,
    name_repo=name_repos,
    stargazers=stargazers_count_list,
    watchers=watchers,
    cuantiles_star=cuantiles_star,
    cuantiles_forks=cuantiles_forks,
    forks=forks_list,
    # used_by=[],
    lenguaje=lenguajes,
    tamaño_kb=sizes,
    issues=open_issues_list,
    network_size=network_count_list
    # pull_request=[],
    # branches=[],
    # no_commits=[]
))

repos_df

,ids,name_repo,stargazers,watchers,cuantiles_star,cuantiles_forks,forks,tamaño_kb,issues,network_size
0,25413198,https://github.com/goburrow/modbus,713,51,"170.25,2.0999,1.4874,1.339","67.25,1.9703,1.5,1.3333",277,110,45,277
1,2037955,https://github.com/rcongiu/Hive-JSON-Serde,702,60,"193.75,1.871,1.4572,1.3289","92.0,1.9783,1.4945,1.3235",395,616,66,395
2,344908284,https://github.com/seatgeek/thefuzz,720,12,"189.0,1.8836,1.5421,1.3115","14.25,1.9298,1.5909,1.3029",57,91,15,57
3,327406312,https://github.com/tailwindlabs/tailwindcss-li...,726,12,"315.0,1.4349,1.302,1.2336","4.75,1.7895,1.4412,1.3061",16,214,0,16
4,69060774,https://github.com/nicklockwood/Expression,713,17,"323.5,1.4992,1.2546,1.1717","10.5,1.9048,1.525,1.3115",40,679,7,40
5,1195935,https://github.com/anru/rsted,701,46,"238.5,1.652,1.4004,1.2705","65.5,2.0153,1.4886,1.3588",279,263,14,279
6,124757468,https://github.com/dreymonde/Delegated,701,12,"457.5,1.1749,1.1391,1.145","13.0,1.5385,1.325,1.2453",33,56,1,33
7,34677782,https://github.com/OWASP/java-html-sanitizer,707,37,"182.75,1.9672,1.5139,1.299","45.5,2.0,1.478,1.3234",179,28260,95,179
8,135138285,https://github.com/Kotlin/kmm-basic-sample,714,58,"163.5,2.3853,1.441,1.2705","33.0,2.0,1.4848,1.3571",135,283,12,135
9,28123883,https://github.com/RigsOfRods/rigs-of-rods,701,72,"164.75,2.0607,1.5096,1.3678","38.5,1.974,1.4803,1.3244",153,218005,160,153


In [ ]:
repos_df[['star_q1','star_gq2','star_gq3','star_gq4']]=repos_df.cuantiles_star.str.split(',',expand=True) 
repos_df.drop(columns=['cuantiles_star'],inplace=True)
repos_df[['fork_q1','fork_gq2','fork_gq3','fork_gq4']]=repos_df.cuantiles_forks.str.split(',',expand=True) 
repos_df.drop(columns=['cuantiles_forks'],inplace=True)

In [ ]:
repos_df.to_csv('../../data/data_sin_etiqueta.csv', index=False)